In [1]:
import tensorflow as tf
import numpy as np
import scipy as sc
import h5py
import time
import matplotlib.pyplot as plt
import math
import seaborn as sns

In [2]:
input_vector_length = 2
hidden_layer_nodes = 2
output_vector_length = 2
learning_rate = 1e-3

In [3]:
#Initialize placeholders , which are variable shape empty objects into which any size tensor can be inputed
Input_layer = tf.placeholder(tf.float32, [None,input_vector_length],name = 'input')
#Output_layer = tf.placeholder(tf.float32, [None,output_vector_length],name = 'output')
Output_vectors = tf.placeholder(tf.float32, [None,output_vector_length],name = 'labels')

In [4]:
#Weights for the hidden layer and biases, Biases not needed for this particular problem
# for input to hidden layer
W1 = tf.Variable(tf.random_normal([input_vector_length,hidden_layer_nodes],stddev=0.1), name='W1')
# for hidden to output layer
W2 = tf.Variable(tf.random_normal([hidden_layer_nodes,output_vector_length],stddev=0.1), name='W2')
#The problem here was I used MSE in a terrible way
#hidden_layer_1 = tf.contrib.layers.fully_connected(Input_layer, output_vector_length, None,biases_regularizer=)

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
#Create the operations on the hidden layer
hidden_layer_1 = tf.matmul(Input_layer,W1)

In [6]:
#Create operations for the output layer
Output_layer = tf.matmul(hidden_layer_1,W2)

In [7]:
#Create Loss function
mse = tf.reduce_mean(tf.square(Output_layer - Output_vectors))

In [8]:
#The Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(mse)
optimizer_real = optimizer.apply_gradients(grads_and_vars,)

In [9]:
#Setup initialization
init_op = tf.global_variables_initializer()

In [10]:
#Code to find angle b/w two vectors
def dotproduct(v1, v2):
  return sum((a*b) for a, b in zip(v1, v2))
def length(v):
  return math.sqrt(dotproduct(v, v))
def angle(v1, v2):
  return math.acos(dotproduct(v1, v2) / (length(v1) * length(v2)))


In [11]:
from scipy.linalg import orth
from scipy.linalg import block_diag
training_points = input_vector_length

Gamma = 0 #Parameter to deviate orthogonal vectors slightly
input_vectors = 10*(np.random.rand(input_vector_length,training_points) - 0.5) #Generate required length of input vectors
input_vectors = orth(input_vectors) + Gamma*np.random.randn(input_vector_length,training_points) #Orthogonalise them and add some random noise
#input_vectors = np.array([[10,0],[17.32,10]]) #If you want to initalize your own training set
#W = np.array([[0.20774353,1.0305219],[-1.2163291,-0.1880631]])
#orthonormal_vectors = np.matmul(W,orthonormal_vectors)
print(input_vectors)

[[-0.66636243  0.745628  ]
 [ 0.745628    0.66636243]]


In [12]:
#Ax = y, We need to invert A => x_train is actually the output of the NN while y_train is the input

A = 10*np.random.randn(input_vector_length,input_vector_length) #Generate a random matrix to invert
x_train = np.matmul(np.linalg.inv(A),input_vectors) # use the generated A and input_vectors to generate x (See Ax = y)
x_validate = np.random.randn(output_vector_length) # Generate a dingle vector for validation
x_train = np.transpose(x_train) #Transpose this for right multiplication
y_train = input_vectors #Set y_train(The one we'll be sending to the feed forward NN) as input_vectors that were initially generated
y_train = np.transpose(y_train) #Keep x_train and y_train consistent
y_validate = np.reshape((np.matmul(A,x_validate)),(1,output_vector_length))#Appropraite shape for the NN
x_validate = np.reshape(x_validate,(1,output_vector_length))#Appropriate shape for the NN

In [13]:
print(x_train)
print(np.shape(x_train))
print(y_train)
print(np.shape(y_train))
print(x_validate)
print(np.shape(x_validate))
print(y_validate)
print(np.shape(y_validate))

[[ 0.12167232 -0.19451165]
 [-1.86510666  1.67452475]]
(2, 2)
[[-0.66636243  0.745628  ]
 [ 0.745628    0.66636243]]
(2, 2)
[[ 0.61072076 -1.82125551]]
(1, 2)
[[-9.46546504 11.56144569]]
(1, 2)


In [14]:
#Some parameters
epochs = 10000
batch_size = 2

In [16]:
with tf.Session() as sess: #Start the session
    #Intialise the variables
    sess.run(tf.global_variables_initializer())
    total_batch = int(training_points/batch_size) # This is used to calculate the average loss in each iteration
    print(total_batch)
    for epoch in range(epochs):
        average_loss = 0 #initialize average_loss as zero
        for i in range(total_batch):
            #x_batch, y_batch = y_train, x_train
            _,c,gradients_and_vars = sess.run([optimizer_real,mse,grads_and_vars],feed_dict = {Input_layer: y_train, Output_vectors: x_train})
#             for g, v in gradients_and_vars:
#                 if g is not None:
#                     print "****************this is variable*************"
#                     print "variable's shape:", v.shape
#                     print v
#                     print "****************this is gradient*************"
#                     print "gradient's shape:", g.shape
#                     print g
            #print(w1,w2)

            average_loss = c/total_batch
            print(sess.run(mse, feed_dict={Input_layer:y_validate , Output_vectors: x_validate}))
        print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(average_loss))



1
1.8618108
('Epoch:', 1, 'cost =', '1.560')
1.8626161
('Epoch:', 2, 'cost =', '1.560')
1.8634424
('Epoch:', 3, 'cost =', '1.559')
1.8642907
('Epoch:', 4, 'cost =', '1.558')
1.8651608
('Epoch:', 5, 'cost =', '1.557')
1.8660536
('Epoch:', 6, 'cost =', '1.556')
1.866969
('Epoch:', 7, 'cost =', '1.556')
1.867907
('Epoch:', 8, 'cost =', '1.555')
1.8688688
('Epoch:', 9, 'cost =', '1.554')
1.8698543
('Epoch:', 10, 'cost =', '1.553')
1.8708637
('Epoch:', 11, 'cost =', '1.552')
1.8718976
('Epoch:', 12, 'cost =', '1.551')
1.8729562
('Epoch:', 13, 'cost =', '1.551')
1.8740399
('Epoch:', 14, 'cost =', '1.550')
1.8751493
('Epoch:', 15, 'cost =', '1.549')
1.8762842
('Epoch:', 16, 'cost =', '1.548')
1.8774457
('Epoch:', 17, 'cost =', '1.547')
1.8786336
('Epoch:', 18, 'cost =', '1.546')
1.8798486
('Epoch:', 19, 'cost =', '1.545')
1.8810909
('Epoch:', 20, 'cost =', '1.544')
1.8823608
('Epoch:', 21, 'cost =', '1.543')
1.8836588
('Epoch:', 22, 'cost =', '1.542')
1.8849852
('Epoch:', 23, 'cost =', '1.541

3.228796
('Epoch:', 237, 'cost =', '0.957')
3.2410533
('Epoch:', 238, 'cost =', '0.953')
3.2533653
('Epoch:', 239, 'cost =', '0.949')
3.2657297
('Epoch:', 240, 'cost =', '0.945')
3.2781475
('Epoch:', 241, 'cost =', '0.941')
3.290618
('Epoch:', 242, 'cost =', '0.936')
3.3031416
('Epoch:', 243, 'cost =', '0.932')
3.3157163
('Epoch:', 244, 'cost =', '0.928')
3.328342
('Epoch:', 245, 'cost =', '0.924')
3.3410206
('Epoch:', 246, 'cost =', '0.920')
3.3537495
('Epoch:', 247, 'cost =', '0.916')
3.3665297
('Epoch:', 248, 'cost =', '0.912')
3.37936
('Epoch:', 249, 'cost =', '0.908')
3.3922408
('Epoch:', 250, 'cost =', '0.904')
3.4051719
('Epoch:', 251, 'cost =', '0.900')
3.4181514
('Epoch:', 252, 'cost =', '0.896')
3.4311805
('Epoch:', 253, 'cost =', '0.892')
3.4442575
('Epoch:', 254, 'cost =', '0.888')
3.4573836
('Epoch:', 255, 'cost =', '0.884')
3.4705565
('Epoch:', 256, 'cost =', '0.880')
3.483778
('Epoch:', 257, 'cost =', '0.876')
3.4970446
('Epoch:', 258, 'cost =', '0.872')
3.5103605
('Epoc

5.970625
('Epoch:', 452, 'cost =', '0.221')
5.9757833
('Epoch:', 453, 'cost =', '0.219')
5.980784
('Epoch:', 454, 'cost =', '0.217')
5.985632
('Epoch:', 455, 'cost =', '0.215')
5.9903183
('Epoch:', 456, 'cost =', '0.213')
5.9948454
('Epoch:', 457, 'cost =', '0.211')
5.9992113
('Epoch:', 458, 'cost =', '0.209')
6.0034156
('Epoch:', 459, 'cost =', '0.207')
6.0074573
('Epoch:', 460, 'cost =', '0.205')
6.0113378
('Epoch:', 461, 'cost =', '0.203')
6.0150504
('Epoch:', 462, 'cost =', '0.201')
6.0186005
('Epoch:', 463, 'cost =', '0.199')
6.021984
('Epoch:', 464, 'cost =', '0.197')
6.025198
('Epoch:', 465, 'cost =', '0.195')
6.0282483
('Epoch:', 466, 'cost =', '0.193')
6.031134
('Epoch:', 467, 'cost =', '0.191')
6.033845
('Epoch:', 468, 'cost =', '0.189')
6.0363865
('Epoch:', 469, 'cost =', '0.187')
6.0387597
('Epoch:', 470, 'cost =', '0.185')
6.040965
('Epoch:', 471, 'cost =', '0.183')
6.0429964
('Epoch:', 472, 'cost =', '0.182')
6.0448503
('Epoch:', 473, 'cost =', '0.180')
6.0465345
('Epoch:

2.925557
('Epoch:', 701, 'cost =', '0.013')
2.907216
('Epoch:', 702, 'cost =', '0.013')
2.8889353
('Epoch:', 703, 'cost =', '0.013')
2.870722
('Epoch:', 704, 'cost =', '0.013')
2.8525696
('Epoch:', 705, 'cost =', '0.012')
2.8344817
('Epoch:', 706, 'cost =', '0.012')
2.8164625
('Epoch:', 707, 'cost =', '0.012')
2.7985039
('Epoch:', 708, 'cost =', '0.012')
2.7806146
('Epoch:', 709, 'cost =', '0.012')
2.7627928
('Epoch:', 710, 'cost =', '0.012')
2.7450387
('Epoch:', 711, 'cost =', '0.012')
2.7273502
('Epoch:', 712, 'cost =', '0.011')
2.7097373
('Epoch:', 713, 'cost =', '0.011')
2.6921911
('Epoch:', 714, 'cost =', '0.011')
2.674716
('Epoch:', 715, 'cost =', '0.011')
2.6573117
('Epoch:', 716, 'cost =', '0.011')
2.6399796
('Epoch:', 717, 'cost =', '0.011')
2.622718
('Epoch:', 718, 'cost =', '0.011')
2.605531
('Epoch:', 719, 'cost =', '0.011')
2.5884173
('Epoch:', 720, 'cost =', '0.010')
2.5713792
('Epoch:', 721, 'cost =', '0.010')
2.5544183
('Epoch:', 722, 'cost =', '0.010')
2.5375328
('Epoc

0.58204514
('Epoch:', 970, 'cost =', '0.001')
0.5801698
('Epoch:', 971, 'cost =', '0.001')
0.5783167
('Epoch:', 972, 'cost =', '0.001')
0.5764863
('Epoch:', 973, 'cost =', '0.001')
0.5746774
('Epoch:', 974, 'cost =', '0.001')
0.57288986
('Epoch:', 975, 'cost =', '0.001')
0.5711236
('Epoch:', 976, 'cost =', '0.001')
0.5693791
('Epoch:', 977, 'cost =', '0.001')
0.5676548
('Epoch:', 978, 'cost =', '0.001')
0.5659526
('Epoch:', 979, 'cost =', '0.001')
0.5642702
('Epoch:', 980, 'cost =', '0.001')
0.56260884
('Epoch:', 981, 'cost =', '0.001')
0.56096756
('Epoch:', 982, 'cost =', '0.001')
0.559345
('Epoch:', 983, 'cost =', '0.001')
0.55774313
('Epoch:', 984, 'cost =', '0.001')
0.55615956
('Epoch:', 985, 'cost =', '0.001')
0.55459595
('Epoch:', 986, 'cost =', '0.001')
0.55305165
('Epoch:', 987, 'cost =', '0.001')
0.5515256
('Epoch:', 988, 'cost =', '0.001')
0.5500186
('Epoch:', 989, 'cost =', '0.001')
0.5485302
('Epoch:', 990, 'cost =', '0.001')
0.54705966
('Epoch:', 991, 'cost =', '0.001')
0.

...

6.771472e-11
('Epoch:', 8677, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8678, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8679, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8680, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8681, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8682, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8683, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8684, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8685, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8686, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8687, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8688, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8689, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8690, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8691, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8692, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8693, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8694, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8695, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8696, 'cost =', '0.000')
6.771472e-11
('Epoch

6.771472e-11
('Epoch:', 8933, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8934, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8935, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8936, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8937, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8938, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8939, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8940, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8941, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8942, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8943, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8944, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8945, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8946, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8947, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8948, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8949, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8950, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8951, 'cost =', '0.000')
6.771472e-11
('Epoch:', 8952, 'cost =', '0.000')
6.771472e-11
('Epoch

4.6171067e-11
('Epoch:', 9204, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9205, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9206, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9207, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9208, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9209, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9210, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9211, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9212, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9213, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9214, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9215, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9216, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9217, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9218, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9219, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9220, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9221, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9222, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9223, 'cost =', '0.000')


4.6171067e-11
('Epoch:', 9473, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9474, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9475, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9476, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9477, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9478, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9479, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9480, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9481, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9482, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9483, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9484, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9485, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9486, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9487, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9488, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9489, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9490, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9491, 'cost =', '0.000')
4.6171067e-11
('Epoch:', 9492, 'cost =', '0.000')


1.3670842e-11
('Epoch:', 9749, 'cost =', '0.000')
1.3670842e-11
('Epoch:', 9750, 'cost =', '0.000')
1.3670842e-11
('Epoch:', 9751, 'cost =', '0.000')
1.3670842e-11
('Epoch:', 9752, 'cost =', '0.000')
1.3670842e-11
('Epoch:', 9753, 'cost =', '0.000')
1.3670842e-11
('Epoch:', 9754, 'cost =', '0.000')
1.3670842e-11
('Epoch:', 9755, 'cost =', '0.000')
1.3670842e-11
('Epoch:', 9756, 'cost =', '0.000')
1.3670842e-11
('Epoch:', 9757, 'cost =', '0.000')
1.3670842e-11
('Epoch:', 9758, 'cost =', '0.000')
1.3670842e-11
('Epoch:', 9759, 'cost =', '0.000')
1.3670842e-11
('Epoch:', 9760, 'cost =', '0.000')
1.3670842e-11
('Epoch:', 9761, 'cost =', '0.000')
1.3670842e-11
('Epoch:', 9762, 'cost =', '0.000')
1.3670842e-11
('Epoch:', 9763, 'cost =', '0.000')
1.3670842e-11
('Epoch:', 9764, 'cost =', '0.000')
1.3670842e-11
('Epoch:', 9765, 'cost =', '0.000')
1.3670842e-11
('Epoch:', 9766, 'cost =', '0.000')
1.3670842e-11
('Epoch:', 9767, 'cost =', '0.000')
1.3670842e-11
('Epoch:', 9768, 'cost =', '0.000')
